In [1]:
import numpy as np
import pandas as pd
# np.set_printoptions(precision = 3, suppress = False)

In [2]:
%%HTML
<style>
   .nodes {
        display: flex;
        justify-content: center;
        background-color: rgb(240, 151, 151);
        width: 12px;
        border-radius: 3px;
        color:black
    }
</style>

## 1.Zadanie
### Vstupne udaje

In [3]:
# Section properties
E = 2.83*(10**7) #kPa
A = 0.06 #m2
I = 4.5*(10**(-4)) #m-4

# lengths
a = 3 #m
b = 3 #m
c = 3 #m

# loads
F = 40 #kN
q = 5 #kN/m

<!-- k01 = np.array([[E*A/l, 0, 0], 
                [0, 12*E*I/l**3, 6*E*I/l**2], 
                [0, 6*E*I/l**2, 4*E*I/l]])
k01 -->

In [4]:
class MemberStiffnessMatrix:

    def localMember(len, A, E, I):
        return pd.DataFrame([[E*A/len, 0, 0], 
                            [0, 12*E*I/len**3, 6*E*I/len**2], 
                            [0, 6*E*I/len**2, 4*E*I/len]])

    def globalMember(rotationMatrix, localMemberStiffnessMatrix, transformationMatrix):
        A = rotationMatrix
        k = localMemberStiffnessMatrix
        B = transformationMatrix
        
        # shape of final output 
        # [kaa, kab],
        # [kba, kbb]
        
        kaa = A.T*A
        kba = B*kaa
        kab = kba.T
        kbb = B*kab
        
        col1 = pd.concat([kaa, kba])
        col2 = pd.concat([kab,kbb])
        
        return pd.concat([col1, col2], axis=1)

In [5]:
class TransformationMatrices:
    def rotation(alfa):
        alfa_rad = np.deg2rad(alfa)
        return pd.DataFrame([[np.cos(alfa_rad), np.sin(alfa_rad), 0], 
                            [-np.sin(alfa_rad), np.cos(alfa_rad), 0], 
                            [0, 0, 1]])

    def transmission(lx, ly):
        return pd.DataFrame([[-1, 0, 0], 
                            [0, -1, 0], 
                            [-ly, lx, -1]])

In [7]:

class BendingMoment:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(length, F):
        return F*length/8
    
    def distributedLoad(length, q):
        return q*length**2/12
    
class Reactions:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(F):
        return F/2
    
    def distributedLoad(length, q):
        return q*length/2

def loadMatrix(N,V,M):
    return pd.DataFrame([[N], 
                         [V], 
                         [M]])



In [8]:
%%HTML
<img src="schemaa.png" width="400" height="400" style="position: relative"/>
<div style="position:absolute; top:350px; left:90px;" class="nodes">1</div>
<div style="position:absolute; top:40px; left:90px;" class="nodes">2</div>
<div style="position:absolute; top:40px; left:340px;" class="nodes">3</div>
<div style="position:absolute; top:350px; left:340px;" class="nodes">4</div>

### Prut 1-2

In [9]:
k01 = MemberStiffnessMatrix.localMember(c,A,E,I)
k01

,0,1,2
0,566000.0,0.0,0.0
1,0.0,5660.0,8490.0
2,0.0,8490.0,16980.0


In [10]:
%%HTML
<img src="rotation-matrix.png" width="400" height="300"/>

In [11]:
alfa = 270 #deg
A0 = TransformationMatrices.rotation(alfa)
A0

,0,1,2
0,-1.836970e-16,-1.000000e+00,0
1,1.000000e+00,-1.836970e-16,0
2,0.000000e+00,0.000000e+00,1


In [12]:
lx = 0 #m
ly = -3 #m

B = TransformationMatrices.transmission (lx,ly)
B

,0,1,2
0,-1,0,0
1,0,-1,0
2,3,0,-1


In [13]:
k1 = MemberStiffnessMatrix.globalMember(A0, k01, B)
k1

,0,1,2,0,1,2
0,3.374460e-32,-1.000000e+00,0.0,-3.374460e-32,-0.000000e+00,0.0
1,-1.000000e+00,3.374460e-32,0.0,-0.000000e+00,-3.374460e-32,0.0
2,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,-1.0
0,-3.374460e-32,-0.000000e+00,0.0,3.374460e-32,-0.000000e+00,0.0
1,-0.000000e+00,-3.374460e-32,0.0,-0.000000e+00,3.374460e-32,0.0
2,0.000000e+00,0.000000e+00,-1.0,0.000000e+00,0.000000e+00,1.0


Local loading matrix

F = (N,V,M)

In [14]:
N = 0 #kN
V = Reactions.distributedLoad(c, q) #kN
M = BendingMoment.distributedLoad(c, q) #kNm


In [15]:
F1_l = loadMatrix(N, V, M)
F2_l = loadMatrix(N, -V, M)
F2_l

,0
0,0.00
1,-7.50
2,3.75


Global loading matrix

In [16]:
F1_g = F1_l*A0.T
F2_g = F2_l*A0.T
pd.concat([F1_g, F2_g]).drop(columns=[1, 2])

,0
0,-0.0
1,-7.5
2,0.0
0,-0.0
1,7.5
2,0.0
